In [2]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
os.getcwd()

'c:\\Users\\kavou\\Downloads'

In [4]:
#import PIT data for 2018-2022 ia a list of dataframes
xls = pd.ExcelFile('2007-2022-PIT-Counts-by-CoC (3).xlsx')

n = 2022-2018+1
PIT_df_list = [pd.DataFrame() for _ in range(n)]

for num,year in enumerate(range(2022,2018-1,-1)):
    PIT_df_list[num] = pd.read_excel(xls, str(year))
    PIT_df_list[num].insert(1, 'Year', year)

In [5]:
#concatenate all dataframes into one
PIT_df = pd.DataFrame()
for df in PIT_df_list:
    try:
        df = df[df['CoC'].notna()]
        PIT_df = pd.concat([PIT_df, df.iloc[:-1]])
    except:
        df = df.rename(columns={'CoC Number': 'CoC'})#some years the area code column is named 'CoC' and some others it's named 'CoC Number'
        df = df[df['CoC'].notna()]
        PIT_df = pd.concat([PIT_df, df.iloc[:-1]])
PIT_df.head()

,CoC,Year,CoC Name,CoC Category,"Overall Homeless, 2022","Overall Homeless - Under 18, 2022","Overall Homeless - Age 18 to 24, 2022","Overall Homeless - Over 24, 2022","Overall Homeless - Female, 2022","Overall Homeless - Male, 2022","Overall Homeless - Transgender, 2022","Overall Homeless - Gender that is not Singularly Female or Male, 2022","Overall Homeless - Gender Questioning, 2022","Overall Homeless - Non-Hispanic/Non-Latino, 2022","Overall Homeless - Hispanic/Latino, 2022","Overall Homeless - White, 2022","Overall Homeless - Black, African American, or African, 2022","Overall Homeless - Asian or Asian American, 2022","Overall Homeless - American Indian, Alaska Native, or Indigenous, 2022","Overall Homeless - Native Hawaiian or Other Pacific Islander, 2022","Overall Homeless - Multiple Races, 2022","Sheltered ES Homeless, 2022","Sheltered ES Homeless - Under 18, 2022","Sheltered ES Homeless - Age 18 to 24, 2022","Sheltered ES Homeless - Over 24, 2022","Sheltered ES Homeless - Female, 2022","Sheltered ES Homeless - Male, 2022","Sheltered ES Homeless - Transgender, 2022","Sheltered ES Homeless - Gender that is not Singularly Female or Male, 2022","Sheltered ES Homeless - Gender Questioning, 2022","Sheltered ES Homeless - Non-Hispanic/Non-Latino, 2022","Sheltered ES Homeless - Hispanic/Latino, 2022","Sheltered ES Homeless - White, 2022","Sheltered ES Homeless - Black, African American, or African, 2022","Sheltered ES Homeless - Asian or Asian American, 2022","Sheltered ES Homeless - American Indian, Alaska Native, or Indigenous, 2022","Sheltered ES Homeless - Native Hawaiian or Other Pacific Islander, 2022","Sheltered ES Homeless - Multiple Races, 2022","Sheltered TH Homeless, 2022","Sheltered TH Homeless - Under 18, 2022","Sheltered TH Homeless - Age 18 to 24, 2022","Sheltered TH Homeless - Over 24, 2022","Sheltered TH Homeless - Female, 2022","Sheltered TH Homeless - Male, 2022","Sheltered TH Homeless - Transgender, 2022","Sheltered TH Homeless - Gender that is not Singularly Female or Male, 2022","Sheltered TH Homeless - Gender Questioning, 2022","Sheltered TH Homeless - Non-Hispanic/Non-Latino, 2022","Sheltered TH Homeless - Hispanic/Latino, 2022","Sheltered TH Homeless - White, 2022","Sheltered TH Homeless - Black, African American, or African, 2022","Sheltered TH Homeless - Asian or Asian American, 2022","Sheltered TH Homeless - American Indian, Alaska Native, or Indigenous, 2022","Sheltered TH Homeless - Native Hawaiian or Other Pacific Islander, 2022","Sheltered TH Homeless - Multiple Races, 2022","Sheltered SH Homeless, 2022","Sheltered SH Homeless - Under 18, 2022","Sheltered SH Homeless - Age 18 to 24, 2022","Sheltered SH Homeless - Over 24, 2022","Sheltered SH Homeless - Female, 2022","Sheltered SH Homeless - Male, 2022","Sheltered SH Homeless - Transgender, 2022","Sheltered SH Homeless - Gender that is not Singularly Female or Male, 2022","Sheltered SH Homeless - Gender Questioning, 2022","Sheltered SH Homeless - Non-Hispanic/Non-Latino, 2022","Sheltered SH Homeless - Hispanic/Latino, 2022","Sheltered SH Homeless - White, 2022","Sheltered SH Homeless - Black, African American, or African, 2022","Sheltered SH Homeless - Asian or Asian American, 2022","Sheltered SH Homeless - American Indian, Alaska Native, or Indigenous, 2022","Sheltered SH Homeless - Native Hawaiian or Other Pacific Islander, 2022","Sheltered SH Homeless - Multiple Races, 2022","Sheltered Total Homeless, 2022","Sheltered Total Homeless - Under 18, 2022","Sheltered Total Homeless - Age 18 to 24, 2022","Sheltered Total Homeless - Over 24, 2022","Sheltered Total Homeless - Female, 2022","Sheltered Total Homeless - Male, 2022","Sheltered Total Homeless - Transgender, 2022","Sheltered Total Homeless - Gender that is not Singularly Female or Male, 2022","Sheltered Total Homeless - Gender Questioning, 2022","Sheltered Total Homeless - Non-Hispanic/Non-Latino, 2022","Sheltered Total Homeless - Hispanic/Latino, 2022","Sheltered Total Homeless - White, 2

In [6]:
PIT_df.iloc[:,4:] = PIT_df.iloc[:,4:].fillna(0)
#After converting NAs to zeros we can easily calculate overall and unsheltered homeless for each year
PIT_df['Overall Homeless'] = PIT_df['Overall Homeless, 2022'] + PIT_df['Overall Homeless, 2021'] + PIT_df['Overall Homeless, 2020'] + PIT_df['Overall Homeless, 2019'] + PIT_df['Overall Homeless, 2018']
PIT_df['Unsheltered Homeless'] = PIT_df['Unsheltered Homeless, 2022'] + PIT_df['Unsheltered Homeless, 2021'] + PIT_df['Unsheltered Homeless, 2020'] + PIT_df['Unsheltered Homeless, 2019'] + PIT_df['Unsheltered Homeless, 2018']
#We want the state totals, so the area code is unnecessary
PIT_df[['State','Code']] = PIT_df['CoC'].str.split("-").to_list()
PIT_df = PIT_df[['State','Year','Overall Homeless','Unsheltered Homeless']]
PIT_df

,State,Year,Overall Homeless,Unsheltered Homeless
0,AK,2022,1494.0,125.0
1,AK,2022,826.0,232.0
2,AL,2022,943.0,342.0
3,AL,2022,585.0,197.0
4,AL,2022,232.0,34.0
...,...,...,...,...
382,WV,2018,108.0,10.0
383,WV,2018,190.0,21.0
384,WV,2018,317.0,57.0
385,WV,2018,628.0,95.0


In [7]:
#Re-arrange table
PIT_df = PIT_df.melt(id_vars=['State','Year'], var_name='Column', value_name='Values')
table = pd.pivot_table(PIT_df, index = 'State', columns = ['Year','Column'], values = 'Values', aggfunc = 'sum', fill_value=0)
table

Year               2018                                  2019  \
Column Overall Homeless Unsheltered Homeless Overall Homeless   
State                                                           
AK                 2016                  309             1907   
AL                 3434                 1010             3261   
AR                 2712                 1257             2717   
AZ                 9865                 4066            10007   
CA               129972                89543           151278   
CO                10857                 4329             9619   
CT                 3976                  581             3033   
DC                 6904                  600             6521   
DE                 1082                   93              921   
FL                31030                13393            28328   
GA                 9499                 3599            10443   
GU                  854                  727              875   
HI                 6530                 3475             6412   
IA                 2749                  241             2315   
ID                 2012                  723             2315   
IL                10643                 1942            10199   
IN                 5258                  579             5471   
KS                 2067                  318             2279   
KY                 3688                  868             4079   
LA                 3059                 1065             2941   
MA                20068                  985            18471   
MD                 7144                 1764             6561   
ME                 2516                   98             2106   
MI                 8351                  899             8575   
MN                 7243                 1121             7977   
MO                 6032                 1253             6281   
MP                 1044                 1022             1807   
MS                 1352                  621             1184   
MT                 1405                  373             1357   
NC                 9268                 2450             9314   
ND                  542                   48              557   
NE                 2421                  144             2365   
NH                 1450                  154             1396   
NJ                 9398                 1623             8862   
NM                 2551                  796             3241   
NV                 7544                 4239             7169   
NY                91897                 4294            92091   
OH                10249                 1315            10345   
OK                 3871                 1191             3944   
OR                14476                 8925            15876   
PA                13512                 1744            13199   
PR                 3182                 2512             2535   
RI                 1101                   51             1055   
SC                 3933                 1344             4172   
SD                 1159                  285              995   
TN                 7883                 2855             7467   
TX                25310                10534            25848   
UT                 2876                  420             2798   
VA                 5975                  915             5783   
VI                  486                  307              314   
VT                 1291                   82             1089   
WA                22304                10621            21577   
WI                 4907                  382             4538   
WV                 1243                  183             1397   
WY                  639                  199              548   

Year                                    2020                       \
Column Unsheltered Homeless Overall Homeless Unsheltered Homeless   
State                                                               
AK                    

In [8]:
table.to_csv('HIC_GAvsUSA_Overall_and_Unsheltered.csv')

In [9]:
#calculate the year-to-year changes in overall and unsheltered homeless numbers
changes_df = table.copy()
changes_df[('2018 to 2019', 'Overall Homeless')] = (changes_df.loc[:,(2019, 'Overall Homeless')]-changes_df.loc[:,(2018, 'Overall Homeless')])/changes_df.loc[:,(2018, 'Overall Homeless')]*100
changes_df[('2019 to 2020', 'Overall Homeless')] = (changes_df.loc[:,(2020, 'Overall Homeless')]-changes_df.loc[:,(2019, 'Overall Homeless')])/changes_df.loc[:,(2019, 'Overall Homeless')]*100
changes_df[('2020 to 2021', 'Overall Homeless')] = (changes_df.loc[:,(2021, 'Overall Homeless')]-changes_df.loc[:,(2020, 'Overall Homeless')])/changes_df.loc[:,(2020, 'Overall Homeless')]*100
changes_df[('2021 to 2022', 'Overall Homeless')] = (changes_df.loc[:,(2022, 'Overall Homeless')]-changes_df.loc[:,(2021, 'Overall Homeless')])/changes_df.loc[:,(2021, 'Overall Homeless')]*100
changes_df[('2018 to 2019', 'Unsheltered Homeless')] = (changes_df.loc[:,(2019, 'Unsheltered Homeless')]-changes_df.loc[:,(2018, 'Unsheltered Homeless')])/changes_df.loc[:,(2018, 'Unsheltered Homeless')]*100
changes_df[('2019 to 2020', 'Unsheltered Homeless')] = (changes_df.loc[:,(2020, 'Unsheltered Homeless')]-changes_df.loc[:,(2019, 'Unsheltered Homeless')])/changes_df.loc[:,(2019, 'Unsheltered Homeless')]*100
changes_df[('2020 to 2021', 'Unsheltered Homeless')] = (changes_df.loc[:,(2021, 'Unsheltered Homeless')]-changes_df.loc[:,(2020, 'Unsheltered Homeless')])/changes_df.loc[:,(2020, 'Unsheltered Homeless')]*100
changes_df[('2021 to 2022', 'Unsheltered Homeless')] = (changes_df.loc[:,(2022, 'Unsheltered Homeless')]-changes_df.loc[:,(2021, 'Unsheltered Homeless')])/changes_df.loc[:,(2021, 'Unsheltered Homeless')]*100
changes_df = changes_df.sort_index(axis=1)
changes_df

Year               2018                                  2019  \
Column Overall Homeless Unsheltered Homeless Overall Homeless   
State                                                           
AK                 2016                  309             1907   
AL                 3434                 1010             3261   
AR                 2712                 1257             2717   
AZ                 9865                 4066            10007   
CA               129972                89543           151278   
CO                10857                 4329             9619   
CT                 3976                  581             3033   
DC                 6904                  600             6521   
DE                 1082                   93              921   
FL                31030                13393            28328   
GA                 9499                 3599            10443   
GU                  854                  727              875   
HI                 6530                 3475             6412   
IA                 2749                  241             2315   
ID                 2012                  723             2315   
IL                10643                 1942            10199   
IN                 5258                  579             5471   
KS                 2067                  318             2279   
KY                 3688                  868             4079   
LA                 3059                 1065             2941   
MA                20068                  985            18471   
MD                 7144                 1764             6561   
ME                 2516                   98             2106   
MI                 8351                  899             8575   
MN                 7243                 1121             7977   
MO                 6032                 1253             6281   
MP                 1044                 1022             1807   
MS                 1352                  621             1184   
MT                 1405                  373             1357   
NC                 9268                 2450             9314   
ND                  542                   48              557   
NE                 2421                  144             2365   
NH                 1450                  154             1396   
NJ                 9398                 1623             8862   
NM                 2551                  796             3241   
NV                 7544                 4239             7169   
NY                91897                 4294            92091   
OH                10249                 1315            10345   
OK                 3871                 1191             3944   
OR                14476                 8925            15876   
PA                13512                 1744            13199   
PR                 3182                 2512             2535   
RI                 1101                   51             1055   
SC                 3933                 1344             4172   
SD                 1159                  285              995   
TN                 7883                 2855             7467   
TX                25310                10534            25848   
UT                 2876                  420             2798   
VA                 5975                  915             5783   
VI                  486                  307              314   
VT                 1291                   82             1089   
WA                22304                10621            21577   
WI                 4907                  382             4538   
WV                 1243                  183             1397   
WY                  639                  199              548   

Year                                    2020                       \
Column Unsheltered Homeless Overall Homeless Unsheltered Homeless   
State                                                               
AK                    

In [10]:
changes_df.to_csv('HIC_GAvsUSA_changes_in_Overall_and_Unsheltered.csv')